In [12]:
import numpy as np
import pandas as pd
import os
import pickle 
import procyclingstats
import power_helper as ph
import plotly.express as px

In [13]:
df = pd.read_csv("Data_edited.csv")
# Riders strikes
df = df.drop(axis=0, index=248) # 1996 stage 9
df = df.drop(axis=0, index=234) # 1995 stage 16
# Calculate power for full tour with fixed grade
total_climbed = df.groupby("year").stage_vertical_meters.sum()
total_distance = df.groupby("year").stage_distance.sum()
gc_weights = df.groupby("year").gc_weight.mean()
gc_time = df.groupby("year").gc_stage_time.sum()
total_grades = total_climbed/(total_distance*1000)
velocity = (total_distance*1000)/gc_time
year_power = ph.cycling_power(total_grades, gc_weights, velocity)

In [1]:
import plotly.graph_objects as go

In [722]:
from geopy import distance, geodesic

In [730]:
import gpxpy


In [768]:
def get_gpx(year, stage_index):
    gpx_file = open('./Routes/{0}/stage-{1}-parcours.gpx'.format(year,stage_index), 'r')

    gpx = gpxpy.parse(gpx_file)
    gpx_dict = dict(lat = [], lon = [], elev = [], dist=[])
    for track in gpx.tracks:
        for segment in track.segments:
            for point_idx in range(1, len(segment.points)):
                # Calculate distance between consecutive points and add to total
                gpx_dict['lat'].append(segment.points[point_idx].latitude)
                gpx_dict['lon'].append(segment.points[point_idx].longitude)
                gpx_dict['elev'].append(segment.points[point_idx].elevation)
                gpx_dict['dist'].append(segment.points[point_idx].distance_2d(segment.points[point_idx - 1]))
        
    df_route = pd.DataFrame(gpx_dict)
    df_route['dist'] = df_route['dist']/1000
    df_route['total_distance'] = df_route.dist.cumsum()
    
    return df_route

In [788]:
px.line(df_route, x='total_distance', y='elev')

In [787]:
import pandas as pd
import requests

fig = px.scatter_mapbox(df_route, lat="lat", lon="lon", hover_name='total_distance', height=600, text='elev', width=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig['data'][0]['mode'] = 'lines+markers'
fig.show()



In [6]:
# Power to speed

# avg_grades =  df.groupby("year").stage_vertical_meters.sum() / (df.groupby("year").stage_distance.sum()*1000) 
avg_speed = ((df.groupby("year").stage_distance.sum()*1000) / df.groupby("year").gc_stage_time.sum()) * 3.6

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df.year.unique(), y=df.groupby("year").power.mean(), name="power"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.year.unique(), y=avg_speed, name="speed"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df.year.unique(), y=year_power, name="year_power")
)

# Add figure title
fig.update_layout(
    title_text="Power and speed by year"
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Power</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Grade</b>", secondary_y=True)
fig.update_yaxes(range = [0,500])
fig.update_yaxes(range = [30,50], secondary_y=True)

fig.show()

In [7]:
px.scatter(color=df.year.unique(), x=df.groupby("year").stage_vertical_meters.sum(), y=df.groupby("year").stage_distance.sum(), title="Distance compared to elevation gain over the years")


In [8]:
px.scatter(df,x="year", y="stage_grade", title="Stage grades per year")


In [11]:

fig = px.scatter(df.groupby("year").power.median(), title="Median stage power compared to full tour power")
fig.add_scatter(x=year_power.index, y=year_power.values)
fig.show()

In [12]:
px.scatter(df[(df.stage_distance > 50) & (df.stage_type != "ITT")], x="year", y="power", hover_data="stage_url", color="stage_distance", trendline="ols")


In [13]:
px.box(df[(df.stage_distance > 50) & (df.stage_type != "ITT")], x="year", y="power", hover_data="stage_url", color="year")